In [ ]:
import os
import sys
from google.colab import drive
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Limpieza de la tabla de datos sociodemográficos**

####**Descripción:**

Filtra y limpia la tabla de datos sociodemográficos eliminando registros de los  beneficiarios que no están presentes en la tabla de seguimiento nutricional, corrigiendo las fechas de nacimiento, imputando datos faltantes a algunas variables codificadas numéricamente y preprocesando toda la tabla para ser utilizada en modelamiento. Genera un nueva tabla de datos con estos cambios y otra preprocesada para modelamiento.
 
####**Requerimientos:**

-  Diccionario `datatypes_dictionaries/dtypes_sociodemo.json`, que mapea cada columna de la tabla de datos sociodemográficos al *datatype* de *Pandas* más adecuado.
- Diccionario `datatypes_dictionaries/cat_soc.json`, que mapea cada columna categórica de la tabla de datos sociodemográficos a una lista (ordenada, si aplica) de sus categorías.
- Diccionario `datatypes_dictionaries/cols_ord.json` que incluye una lista de las variables categóricas ordinales de cada tabla de datos.
- Tabla de relación `auxiliary_data/IdBeneficiario_Id.parquet` que relaciona los identificadores de las tablas de datos nutricionales (`IdBeneficiario`) y sociodemográficos (`Id`).
- Tabla de datos sociodemográficos preprocesada con identificadores asignados correctamente `clean_data/sociodemo_ids.parquet`.
- Tabla de datos de los beneficiarios con tomas nutricionales preprocesada `clean_data/beneficiario.parquet`.
 
####**Salidas:**
- Tabla de datos `clean_data/sociodemo.parquet` con datos sociodemográficos filtrada para incluir solo los beneficiarios que tienen tomas nutricionales, con fechas corregidas, datos faltantes asignados y categorías ordenadas.
- Tabla de datos `clean_data/sociodemo_beneficiario.parquet`, cruzada con la tabla de datos del beneficiario.
- Tabla de datos `clean_data/sociodemo_model.parquet` como la anterior, pero con los datos faltantes convertidos a 0, las variables categóricas ordinales en formato numérico y las variable categóricas no ordinales (excepto el código de municipio `cod_mpio`) con codificación *one-hot*, para facilitar su uso en modelado.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json

### **Lectura de archivos de datos**

In [ ]:
t0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/cat_soc.json', 'r') as file:
  cat_soc = json.load(file)
# Lee la tabla de datos sociodemográficos preprocesada con los identificadores correctos
soc = pd.read_parquet('clean_data/sociodemo_ids.parquet').sort_values('IdBeneficiario')
for col in soc.columns:
  if dtypes_soc[col] == 'category' and str(soc[col].dtype)[:5] == 'float':
    soc[col] = soc[col].astype('UInt8')
soc = soc.astype(dtypes_soc)
soc.info()
# Lee la tabla de datos de los beneficiarios limpia
tom = pd.read_parquet('clean_data/beneficiario.parquet')
# Tabla de relación para verificar la correspondencia entre identificadores
ids = pd.read_parquet('auxiliary_data/IdBeneficiario_Id.parquet')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1508837 entries, 2028667 to 3327996
Data columns (total 68 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Cod_clase                  1508837 non-null  category      
 1   Ind_grupo_sisben_4         1508837 non-null  category      
 2   Ind_nivel_sisben_4         1508837 non-null  category      
 3   Tip_vivienda               1508837 non-null  category      
 4   Ind_tiene_energia          1508837 non-null  category      
 5   Ind_tiene_alcantarillado   1508837 non-null  category      
 6   Ind_tiene_gas              1508837 non-null  category      
 7   Ind_tiene_recoleccion      1508837 non-null  category      
 8   Ind_tiene_acueducto        1508837 non-null  category      
 9   n_hogares_vivienda         1508837 non-null  UInt8         
 10  Tip_ocupa_vivienda         1508837 non-null  category      
 11  Ind_tiene_cocina           1508

In [ ]:
# Lista de variables categóricas ordinales
cols_cat = [col for col in soc.columns if col in cat_soc.keys()]
# Lista de variables categóricas ordinales
with open('datatypes_dictionaries/cols_ord.json', 'r') as file:
  cols_ord = json.load(file)['soc']
# Lista de columnas que pueden cruzarse con la tabla de datos del beneficiario
cols_tom = ['FechaNacimiento', 'Sexo', 'Cod_clase', 'Ind_ninguna_discapac']

## **Limpieza de la tabla de datos**

In [ ]:
# Todos los pares 'Id', 'IdBeneficiario' están bien asignados
join = soc[['Id', 'IdBeneficiario']].merge(ids, on='Id', how='left')
(join['IdBeneficiario_x'] != join['IdBeneficiario_y']).sum()

0

In [ ]:
# Cantidad de datos faltantes (NaNs) por columna
nulos = soc.isna().sum()
nulos[nulos > 0]

Sexo                       1
Ind_acudio_salud           5
Ind_fue_atendido_salud    12
dtype: int64

 ### **Validación de categorías**

In [ ]:
# Forzar categorías y ordenar las categorías de variables ordinales
for col in cols_cat:
  categorias = cat_soc[col]
  ordinal = col in cols_ord
  soc[col] = soc[col].cat.set_categories(categorias, ordered = ordinal)

In [ ]:
# Cantidad de datos faltantes (NaNs) por columna
nulos = soc.isna().sum()
nulos[nulos > 0]

Sexo                            1
Ind_acudio_salud          1338701
Ind_fue_atendido_salud    1363609
Ind_recibe_comida          438989
Ind_leer_escribir         1069848
Nivel_educ_padre           577509
Nivel_educ_madre            68813
dtype: int64

### **Asignación de valores a partir la tabla de datos de beneficiarios**

In [ ]:
# Reemplaza algunas categorías en la tabla de datos sociodemográficos para homologarlas 
# a las presentes en la tabla de datos nutricionales
tom['Sexo'] = tom['Sexo'].replace({
    'F': 0,
    'M': 1
    }).astype('category')
tom['Cod_clase'] = tom['ZonaUbicacion'].replace({
    'CABECERA': 1,
    'RESTO': 2
    }).astype('UInt8').astype('category').cat.set_categories([1, 2, 3])
tom['Ind_ninguna_discapac'] = tom['PresentaDiscapacidad'].replace({
    'SI': 0,
    'NO': 1
    }).astype('category')
# Para las columnas 'FechaNacimiento', 'Sexo', 'Cod_clase' y 'Ind_ninguna_discapac'
tom_min = tom[['IdBeneficiario'] + cols_tom]
soc = soc.merge(tom_min, on='IdBeneficiario', how='left')
for col in cols_tom:
  diferencias = (soc[f'{col}_x'] != soc[f'{col}_y']).isna()
  nan_inicio = soc[f'{col}_x'].isna()
  nan_resueltos = ~soc[f'{col}_y'].isna() & nan_inicio
  soc[f'{col}_x'] = soc[f'{col}_y']
  soc = soc.drop(columns=[f'{col}_y'])
  soc = soc.rename(columns={f'{col}_x': col})
  print(f"En '{col}': {diferencias.sum()} diferencias, {nan_resueltos.sum()} NaNs resueltos de " +
        f"{nan_inicio.sum()}: : {time.time() - t0:.2f} s.")

En 'FechaNacimiento': 0 diferencias, 0 NaNs resueltos de 0: : 21.40 s.
En 'Sexo': 0 diferencias, 1 NaNs resueltos de 1: : 21.50 s.
En 'Cod_clase': 0 diferencias, 0 NaNs resueltos de 0: : 21.59 s.
En 'Ind_ninguna_discapac': 0 diferencias, 0 NaNs resueltos de 0: : 21.67 s.


 ### **Guardado**

In [ ]:
soc.to_parquet('clean_data/sociodemo.parquet')
print(f"Archivo 'sociodemo.parquet' guardado: {time.time() - t0:.2f} s.")

Archivo 'sociodemo.parquet' guardado: 24.93 s.


## **Cruce con tabla de datos de beneficiarios**

Se incluyen variables provenientes de la tabla de datos nutricionales que caracterizan a cada beneficiario.

In [ ]:
# Variables de la tabla de datos de beneficiario que aún no han sido incluidas
tom = tom[[
  'IdBeneficiario', 
  'PaisNacimiento',
  'PesoAlNacer',
  'TallaAlNacer',
  'EdadGestacionalAlNacer',
  'GrupoEtnico',
  'MesesLactanciaMaternaExclusiva',
  'MesesLactanciaMaternaTotal']]
# Tabla de datos combinando toda la información
soc = soc.merge(tom, on='IdBeneficiario', how='left')
# Fracción de datos faltantes (NaNs) por columna
nulos = soc.isna().mean()
nulos[nulos > 0]

Ind_acudio_salud                  0.887240
Ind_fue_atendido_salud            0.903748
Ind_recibe_comida                 0.290945
Ind_leer_escribir                 0.709055
Nivel_educ_padre                  0.382751
Nivel_educ_madre                  0.045607
PaisNacimiento                    0.005896
PesoAlNacer                       0.944151
TallaAlNacer                      0.944481
EdadGestacionalAlNacer            0.995402
GrupoEtnico                       0.011333
MesesLactanciaMaternaExclusiva    0.161848
MesesLactanciaMaternaTotal        0.161848
dtype: float64

### **Guardado**

In [ ]:
soc.to_parquet('clean_data/sociodemo_beneficiario.parquet')
print(f"Archivo 'sociodemo_beneficiario.parquet' guardado: {time.time() - t0:.2f} s.")

Archivo 'sociodemo_beneficiario.parquet' guardado: 31.26 s.


## **Preprocesamiento para modelamiento**
Las variables `PesoAlNacer`, `TallaAlNacer` y `EdadGestacionalAlNacer` se excluyen por tener demasiados datos faltantes (> 90%).
Se crea la variable `Ind_nac_Col` que indica si el país de nacimiento es o no Colombia, y se descarta la variable `PaisNacimiento` por tener demasiadas categorías.

Todas las variables categóricas no ordinales, excepto el municipio, se codifican mediante *one-hot encoding*, generando una tabla de 140 columnas.

Algunos datos faltantes se escriben como cero, si tiene sentido.

In [ ]:
# Año de la encuesta desde 2017 (2017 = 0)
soc['anno_encuesta_2017'] = soc['anno_encuesta'].cat.codes
# Reduce la variable de nacionalidad a si el estudiante es o no extranjero
soc['Ind_nac_Col'] = (soc['PaisNacimiento'] == 'Colombia').astype('UInt8')
# Eliminar columnas que no tiene sentido utilizar en los modelos
soc = soc.drop(columns=[
  'Ind_grupo_sisben_4',
  'FechaNacimiento',
  'anno_encuesta',
  'PesoAlNacer',
  'TallaAlNacer',
  'EdadGestacionalAlNacer',
  'PaisNacimiento'])
# Variables categóricas no ordinales codificadas por números (excepto 'cod_mpio')
cols_tipo = [col for col in soc.columns if col[:3] == 'Tip'] + \
  ['Cod_clase', 'Genera_ingresos', 'cod_dpto', 'GrupoEtnico']
# One-hot encoding
for col in cols_tipo:
  soc = pd.concat([soc, pd.get_dummies(soc[col], prefix=col)], axis='columns')
  soc = soc.drop(columns=[col])
# Columnas en las que el NaN puede interpretarse como 0
cols_nan = ['Ind_acudio_salud',
            'Ind_fue_atendido_salud',
            'Ind_recibe_comida',
            'Ind_leer_escribir',
            'Nivel_educ_padre',
            'Nivel_educ_madre']
for col in cols_nan:
  soc[col] = soc[col].fillna(0)
# Convierte todas las columnas categóricas (excepto 'anno_encuesta) en tipo entero pequeño
for col in soc.columns:
  if col in cat_soc.keys():
    soc[col] = soc[col].astype('UInt8')
 
# Fracción de datos faltantes (NaNs) por columna
nulos = soc.isna().mean()
nulos[nulos > 0]

MesesLactanciaMaternaExclusiva    0.161848
MesesLactanciaMaternaTotal        0.161848
dtype: float64

In [ ]:
# Fracción de datos faltantes (NaNs) por columna
nulos = soc.isna().mean()
nulos[nulos > 0]

MesesLactanciaMaternaExclusiva    0.161848
MesesLactanciaMaternaTotal        0.161848
dtype: float64

In [ ]:
list(soc.columns)

['Ind_nivel_sisben_4',
 'Ind_tiene_energia',
 'Ind_tiene_alcantarillado',
 'Ind_tiene_gas',
 'Ind_tiene_recoleccion',
 'Ind_tiene_acueducto',
 'n_hogares_vivienda',
 'Ind_tiene_cocina',
 'Ind_tiene_nevera',
 'Ind_tiene_lavadora',
 'Ind_tiene_pc',
 'Ind_tiene_internet',
 'n_personas_hogar',
 'Sexo',
 'Ind_ninguna_discapac',
 'Ind_acudio_salud',
 'Ind_fue_atendido_salud',
 'Ind_recibe_comida',
 'Ind_leer_escribir',
 'Ind_estudia',
 'privado_logro_educ',
 'privado_analfabe',
 'privado_inasistencia',
 'privado_rezago',
 'privado_primera_inf',
 'privado_trabajo_inf',
 'privado_desemp_largo',
 'privado_trabajo_info',
 'privado_aseguram',
 'privado_acceso_salud',
 'privado_agua',
 'privado_excreta',
 'privado_suelo',
 'privado_pared',
 'privado_hacina',
 'n_privaciones',
 'IndicadorPobrezaMulti',
 'ingresos_promP_imp',
 'gasto_ppers_imp',
 'gasto_alim_ppers_imp',
 'porc_gasto_alim',
 'union_temprana',
 'n_ninos',
 'Jefat_fem',
 'Nivel_educ_padre',
 'Nivel_educ_madre',
 'Uni_dias_agua',
 'Ind_

### **Guardado**

In [ ]:
# Guarda el dataframe preprocesado en un archivo parquet
soc.to_parquet('clean_data/sociodemo_model.parquet')
print(f"Archivo 'sociodemo_model.parquet' guardado: {time.time() - t0:.2f} s.")

Archivo 'sociodemo_model.parquet' guardado: 40.54 s.
